# Softmax Regression

In [1]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import classification_report

In [2]:
df_train = pd.read_csv('datasets/customer_seg_train.csv', index_col='ID')
df_train.dropna(inplace=True)
df_train.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
ID,,,,,,,,,,
462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
461319,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6,C
460156,Male,No,32,Yes,Healthcare,1.0,Low,3.0,Cat_6,C


In [3]:
df_train['Ever_Married'] = df_train['Ever_Married'].map({'Yes': 1, 'No': 0})
df_train['Graduated'] = df_train['Graduated'].map({'Yes': 1, 'No': 0})
ohe_columns = pd.get_dummies(df_train[['Gender', 'Profession', 'Spending_Score']], prefix=['Gender', 'Profession', 'Spending_Score'], dtype='float')
df_train.drop(columns=['Gender', 'Profession', 'Spending_Score', 'Segmentation'], inplace=True)
df_train = pd.concat([df_train, ohe_columns], axis='columns')
df_train.head()

,Ever_Married,Age,Graduated,Work_Experience,Family_Size,Var_1,Gender_Female,Gender_Male,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,Profession_Lawyer,Profession_Marketing,Spending_Score_Average,Spending_Score_High,Spending_Score_Low
ID,,,,,,,,,,,,,,,,,,,,
462809,0,22,0,1.0,4.0,Cat_4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
466315,1,67,1,1.0,1.0,Cat_6,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
461735,1,67,1,0.0,2.0,Cat_6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
461319,1,56,0,0.0,2.0,Cat_6,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
460156,0,32,1,1.0,3.0,Cat_6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
X_train = df_train.drop(columns=['Var_1'])
X_train

,Ever_Married,Age,Graduated,Work_Experience,Family_Size,Gender_Female,Gender_Male,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,Profession_Lawyer,Profession_Marketing,Spending_Score_Average,Spending_Score_High,Spending_Score_Low
ID,,,,,,,,,,,,,,,,,,,
462809,0,22,0,1.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
466315,1,67,1,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
461735,1,67,1,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
461319,1,56,0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
460156,0,32,1,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463002,1,41,1,0.0,5.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
464685,0,35,0,3.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
465406,0,33,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [5]:
df_test = pd.read_csv('datasets/customer_seg_test.csv', index_col='ID')
df_test.dropna(inplace=True)
df_test['Ever_Married'] = df_test['Ever_Married'].map({'Yes': 1, 'No': 0})
df_test['Graduated'] = df_test['Graduated'].map({'Yes': 1, 'No': 0})
ohe_columns = pd.get_dummies(df_test[['Gender', 'Profession', 'Spending_Score']], prefix=['Gender', 'Profession', 'Spending_Score'], dtype='float')
df_test.drop(columns=['Gender', 'Profession', 'Spending_Score'], inplace=True)
df_test = pd.concat([df_test, ohe_columns], axis='columns')
X_test = df_test.drop(columns=['Var_1'])

In [6]:
mm  = MinMaxScaler()
mm.fit(X_train)
X_train = mm.transform(X_train)
X_test = mm.transform(X_test)

### Softmax Regression using CategoricalCrossEntropy
Using this loss function, the target variables are one hot encoded

In [7]:
target_train_ohe = pd.get_dummies(df_train[['Var_1']], dtype='float')
target_train_ohe.head()

,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
ID,,,,,,,
462809,0.0,0.0,0.0,1.0,0.0,0.0,0.0
466315,0.0,0.0,0.0,0.0,0.0,1.0,0.0
461735,0.0,0.0,0.0,0.0,0.0,1.0,0.0
461319,0.0,0.0,0.0,0.0,0.0,1.0,0.0
460156,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
y_train = target_train_ohe.values
y_test = pd.get_dummies(df_test[['Var_1']], dtype='float').values
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
total_classes = y_test.shape[1]
model = tf.keras.Sequential([
    tf.keras.Input(shape= (X_train.shape[1],) ),
    tf.keras.layers.Dense(units=total_classes, kernel_regularizer=tf.keras.regularizers.L2(0.01))
])
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.SGD(learning_rate=1.), metrics=['accuracy'])
model.fit(
    X_train,
    y_train,
    batch_size=10000,
    epochs=100)

Epoch 1/100


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0725 - loss: 2.2040
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.6611 - loss: 1.3575
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.6716 - loss: 1.2802
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.6716 - loss: 1.2584
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.6716 - loss: 1.2417
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.6716 - loss: 1.2279
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.6716 - loss: 1.2163
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.6716 - loss: 1.2064
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.6716 - loss: 1.1978
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step - accuracy: 0.6716 - loss: 1.1902
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - accuracy: 0.6716 - loss: 1.1835
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 0.6716 - loss: 1.1

In [10]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Loss (Mean error per testing example):', loss)
print('Accuracy: ', accuracy)

Loss (Mean error per testing example): 1.093908667564392
Accuracy:  0.6597028970718384


### Softmax Regression using SparseCategoricalCrossEntropy
Using this loss function, target variables are encoded with integers.

In [11]:
le = LabelEncoder()
y_train2 = le.fit_transform(df_train[['Var_1']])
y_test2 = le.fit_transform(df_test[['Var_1']])
y_train2

c:\Users\chaud\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\chaud\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([3, 5, 5, ..., 5, 5, 3])

In [12]:
model2 = tf.keras.Sequential([
    tf.keras.Input(shape= (X_train.shape[1],) ),
    tf.keras.layers.Dense(units=total_classes, kernel_regularizer=tf.keras.regularizers.L2(0.01))
])
model2.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.SGD(learning_rate=1.), metrics=['accuracy'])
model2.fit(
    X_train,
    y_train2,
    batch_size=10000,
    epochs=100)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2203 - loss: 1.8626
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.6750 - loss: 1.2949
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.6732 - loss: 1.2545
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 0.6722 - loss: 1.2337
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.6717 - loss: 1.2183
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.6716 - loss: 1.2063
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6716 - loss: 1.1967
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.6716 - loss: 1.1887
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.6716 - loss: 1.1819
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.6716 - loss: 1.1760
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.6716 - loss: 1.1708
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.6716 -

In [13]:
loss2, accuracy2 = model2.evaluate(X_test, y_test2, verbose=0)
print('Loss (Mean error per testing example):', loss2)
print('Accuracy: ', accuracy2)

Loss (Mean error per testing example): 1.0953725576400757
Accuracy:  0.6597028970718384


In [14]:
logit = model.predict(X_test, verbose=0) # This is the raw answer of equation wx+b
print('logit is:\n', logit, '\n')
# To convert this to probability generated by sigmoid function, use this command:
y_pred = np.array(tf.nn.softmax(logit))
print('y_pred is:\n', y_pred, '\n')

logit is:
 [[-1.1878986  -0.26748317  0.6184461  ... -1.4546578   2.3169053
  -0.82958204]
 [-1.2447065   0.0347648   0.42249224 ... -1.5603416   2.4725623
  -0.7631763 ]
 [-1.1382535  -0.2834214   0.14119712 ... -1.3493505   2.6632676
  -0.83731645]
 ...
 [-1.1946322  -0.04797053  0.44724086 ... -1.3286798   2.3375845
  -0.76602745]
 [-1.2147753  -0.15540743  0.20538536 ... -1.4629427   2.734739
  -0.76417786]
 [-1.2431585   0.21095389  0.36828417 ... -1.4242108   2.4449215
  -0.81169355]] 

y_pred is:
 [[0.01884658 0.04731118 0.11474086 ... 0.01443379 0.62711835 0.02696792]
 [0.01656983 0.05956423 0.08777552 ... 0.01208481 0.68187934 0.02681909]
 [0.01632747 0.03838553 0.05869177 ... 0.01322028 0.7309681  0.02206044]
 ...
 [0.01967712 0.06193701 0.10162903 ... 0.01720859 0.6729513  0.03020662]
 [0.01451009 0.04185503 0.06003964 ... 0.0113212  0.7532209  0.02276995]
 [0.01719955 0.07362577 0.0861703  ... 0.01435116 0.68743193 0.02647888]] 



### Classification Report

In [15]:
y_pred_classes = np.argmax(y_pred, axis=1)
cr = classification_report(y_test2, y_pred_classes, target_names=[f'Cat {i+1}' for i in range(total_classes)])
print('Classification Report\n\n', cr)

Classification Report

               precision    recall  f1-score   support

       Cat 1       0.00      0.00      0.00        27
       Cat 2       0.00      0.00      0.00       116
       Cat 3       0.00      0.00      0.00       195
       Cat 4       0.00      0.00      0.00       320
       Cat 5       0.00      0.00      0.00        23
       Cat 6       0.66      1.00      0.79      1421
       Cat 7       0.00      0.00      0.00        52

    accuracy                           0.66      2154
   macro avg       0.09      0.14      0.11      2154
weighted avg       0.44      0.66      0.52      2154



c:\Users\chaud\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\chaud\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\chaud\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
